In [ ]:
import os
import spikeinterface.core as si
import spikeinterface.widgets as sw
import spikeinterface.curation as scur
from spikeinterface.comparison import compare_two_sorters


n_cpus = os.cpu_count()
n_jobs = n_cpus - 2

job_kwargs = dict(chunk_duration="1s", n_jobs=n_jobs, progress_bar=True)



In [ ]:
# LOAD SORTING ANALYZER
# Set the path of where the sorting analyzer was created. It should exists within the "parentSortingFolder". Binary recording will exists in the same parent folder
sorter_analyzer_folder = "M:\\AlbusSorting_Results\\exp2024-04-03_sorting\\exp2024-04-03-sess-0_3-raw-D1-385\\exp2024-04-03-sess-0_3-raw-D1-385_MS5s2ch0th4"
sorting_analyzer = si.load_sorting_analyzer(sorter_analyzer_folder, load_extensions=True)


# PRINT SOME GENERAL UNITS INFORMATION
print('number of units : ', sorting_analyzer.get_num_units())
print('total number of spikes : ',sorting_analyzer.sorting.count_total_num_spikes())

print('unit IDs : ', sorting_analyzer.unit_ids)
print('spikes per unit :')
for u, n in sorting_analyzer.sorting.count_num_spikes_per_unit().items():
    print('\tunit "{}" : {}\n'.format(u, n))

print('Current Sorting Analyzer contains the following extensions:')
for e in sorting_analyzer.get_saved_extension_names():
    print('\t{}'.format(e))


print('Sorting properties : ')
for p, v in sorting_analyzer.sorting._properties.items():
    print('\t{} : {}'.format(p, v))


In [ ]:
# Quality metrics as dataframe
quality_metrics_pd = sorting_analyzer.get_extension('quality_metrics').get_data()
quality_metrics_pd

In [ ]:
# PLOT A GENERAL OVERVIEW OF THE UNITS
%matplotlib ipympl
sw.plot_unit_locations(sorting_analyzer, backend="ipywidgets")
sw.plot_all_amplitudes_distributions(sorting_analyzer, figsize=(10,4))
sw.plot_amplitudes(sorting_analyzer, plot_histograms=True, backend="ipywidgets")

In [ ]:
# Compare the sorter with itself to point to similar units
comparison = compare_two_sorters(sorting_analyzer.sorting, sorting_analyzer.sorting, delta_time=1)

%matplotlib ipympl
sw.plot_agreement_matrix(comparison)

In [ ]:
# Explore individual neurons:
%matplotlib ipympl
unit_id = 1
sw.plot_unit_summary(sorting_analyzer, unit_id=unit_id)

In [ ]:
# Compare a pair (or more) neurons
unit_id_1 = 1
unit_id_2 = 1 # Against itself

# Add as many as you want:
units2explore = [unit_id_1, unit_id_2] 

%matplotlib ipympl
for u in units2explore:
    sw.plot_unit_summary(sorting_analyzer, unit_id=u)

sw.plot_crosscorrelograms(sorting_analyzer,  unit_ids=units2explore)
sw.plot_unit_templates(sorting_analyzer, unit_ids=units2explore)
sw.plot_template_metrics(sorting_analyzer,  unit_ids=units2explore)
sw.plot_template_similarity(sorting_analyzer,  unit_ids=units2explore, display_diagonal_values=True)

In [ ]:
# TODO: how to merge or split using spikeinterface

In [ ]:
# If there is no need to merge units, apply Labels manually : value must be a list of labels for all the sorted units 
sorting_analyzer.sorting.set_property(key ='quality', value=['mua'])
print(sorting_analyzer.sorting._properties)

In [ ]:
# Apply curation dict to a Sorting or a SortingAnalyzer
# Use this method only if there is units to merge
#
# Steps are done in this order:
#  1)      Apply removal using curation_dict[“removed_units”]
#  2)      Apply merges using curation_dict[“merge_unit_groups”]
#  3)      Set labels using curation_dict[“manual_labels”]


# Sample Dictionary to be applied on the result to have a “clean” result
curation_dict = dict(
    format_version = "1",
    # Define LABELS
    # For label category with exclusive=True : a column is created and values are the unique label.
    # For label category with exclusive=False : one column per possible is created and values are boolean.
    label_definitions = dict(
        quality = dict(label_options = ["good", "noise", "mua", "artifact"], exclusive = True)
        # Keep adding custom labels:
        # putative_type = dict(label_options = ["excitatory", "inhibitory", "pyramidal", "mitral"], exclusive = False)
    ),
    # 
    unit_ids = sorting_analyzer.unit_ids,
    removed_units = [], # List of units to remove. Example: [31, 42]
    merge_unit_groups = [[], []], # List of Lists of units to merge (at least 2 units require). Example: [[3, 6], [10, 14, 20]]
    manual_labels = [
        dict(unit_id = 1, quality = ["mua"]),
        # Keep adding neurons' labels. Example:
        # dict(unit_id = 2, quality = ["noise"], putative_type = ["excitatory", "pyramidal"])
    ],
)

# Might be some errors while trying to add labels. Comments have been posted to the spikeinterface github 
scur.apply_curation(sorting_or_analyzer=sorting_analyzer, 
    curation_dict = curation_dict,
    censor_ms=0.25,
    new_id_strategy="append",
    merging_mode="soft",
    sparsity_overlap=0.75,
    verbose=False,
    **job_kwargs
    )

In [ ]:
# Save curated sorter analyzer
# Add the suffix 
sorting_analyzer.save_as(format="binary_folder", folder=sorter_analyzer_folder+'_curated')